<h1 style="color:black" align="center">Градиентный бустинг</h1>

Ранее мы изучили бэггинг и случайные леса — подходы к построению композиций, которые независимо обучают каждый базовый алгоритм по некоторому подмножеству обучающих данных. При этом возникает ощущение, что мы используем возможности объединения алгоритмов не в полную силу, и можно было бы строить их так, чтобы каждая следующая модель исправляла ошибки предыдущих. Ниже мы рассмотрим метод, который реализует эту идею — градиентный бустинг, предложенный Фридманом [1]. Он работает для любых дифференцируемых функций потерь и является одним из наиболее мощных и универсальных на сегодняшний день.

<h1 style="color:#008B8B">1. Бустинг в задаче регрессии</h1>

Рассмотрим задачу минимизации квадратичного функционала:

$\large \frac{1}{\ell}
\sum\limits_{i = 1}^{\ell}
    (a(x_i) - y_i)^2
\to
\underset{a}{\text{min}}$

Будем искать итоговый алгоритм в виде суммы *базовых моделей* (weak learners) $b_n(x)$:

$\large a_N(x)
=
\sum\limits_{n = 1}^{N}
    b_n(x)$

где базовые алгоритмы $b_n$ принадлежат некоторому семейству $\mathbb{A}$. Построим первый базовый алгоритм по всем базовым моделям $b \in \mathbb{A}$:

$\large a_1(x) = b_1(x)
:=
\underset{b \in \mathbb{A}}{\text{argmin}}
    \frac{1}{\ell}
    \sum\limits_{i = 1}^{\ell}
        (b(x_i) - y_i)^2$
        
Посчитаем насколько базовая модель ошибается на каждом объете:

$\large s_i^{(1)} = y_i - a_1(x_i)$ - сдвиги


Если прибавить эти остатки к ответам построенного алгоритма, то он не будет допускать ошибок на обучающей выборке. Значит, будет разумно построить второй алгоритм так, чтобы его ответы были как можно ближе к остаткам:

$\large b_2(x)
:=
\underset{b \in \mathbb{A}}{\text{argmin}}
    \frac{1}{\ell}
    \sum\limits_{i = 1}^{\ell}
        (b(x_i) - s_i^{(1)})^2$
        
Если $b_2$ идеальна, тогда мы берем копозицию из двух моделей и она будет всюду выдавать правиьный овет $y_i$:

$\large a_2(x_i) = b_1(x_i) + s_i^{(1)} = b_1(x_i) + y_i - a_1(x_i) = y_i$

Так как зачастую этого недостаточно, тогда мы посчитаем следующие сдвиги:

$\large s_i^{(2)} = y_i - a_2(x_i)$ - сдвиги

Каждый следующий алгоритм тоже будем настраивать на остатки предыдущих:


$\large s_i^{(N)} = 
y_i - \sum\limits_{n = 1}^{N - 1} b_n(x_i)
=
y_i
-
a_{N - 1}(x_i)
\qquad
i = 1, \dots, \ell$

$\large b_N(x)
:=
\underset{b \in \mathbb{A}}{\text{argmin}}
    \frac{1}{\ell}
    \sum\limits_{i = 1}^{\ell}
        (b(x_i) - s_i^{(N)})^2$

<h1 style="color:#008B8B">2. Градиентный бустинг</h1>

Пусть дана некоторая дифференцируемая функция потерь $L(y, z)$. Будем считать, что мы обучили первую модель $b_1(x)$. Как построить следующие модели? Обучение $b_N$ запишем следующую задачу:

$\large \sum\limits_{i = 1}^{\ell} L(y_i, a_{N-1}(x_i) + b_N(x_i)) \to \underset{b_N(x)}{\text{min}}$

Где $a_{N-1}(x_i) = \sum\limits_{n=1}^{N-1} b_n(x)$ композиция из построенной $N-1$ модели.

В этой задаче, мы хотим найти $b_N$ модель такой, что если ее прибавить к построенной композиции, тогда должна получиться более маленькая ошибка. 

Если фукнция потерь удовлетворяет тому, что можно перенести агрумент из одной части в другую $L(y, a+b) = L(y-a, b)$, тогда мы можем дообучать базовую модель на новую целевую переменную:

$\large \sum\limits_{i = 1}^{\ell} L(y_i - a_{N-1}(x_i), b_N(x_i)) \to \underset{b_N(x)}{\text{min}}$, тогда мы получаем задачу обучения базовой модели с обновленной целевой переменной, которая равна разности. 

Но, не все функции потерь способны удовлетворять этому требованию. Поэтому, нам необходим другой подход.

### Другой подход

Ответим в первую очередь на следующий вопрос: если бы в качестве алгоритма $b_N(x)$ мы могли выбрать совершенно любую функцию, то какие значения ей следовало бы принимать на объектах обучающей выборки? Иными словами, нам нужно понять, какие числа $s_1, \ldots, s_{\ell}$ надо выбрать для решения следующей задачи:

$\large \sum\limits_{i = 1}^{\ell}
    L(y_i, a_{N - 1}(x_i) + s_i)
\to
\underset{s_1, \dots, s_\ell}{\text{min}}$


Понятно, что можно требовать $s_i = y_i - a_{N - 1}(x_i)$, но такой подход никак не учитывает особенностей функции потерь $L(y, z)$ и требует лишь точного совпадения предсказаний и истинных ответов. Наприме, имеется не симметричная фукнция потерь, которая по разному штрафует за отклонения, за перепрогноз штраф растет быстрее. Тогда при равной аболютной ошибке штрафы для объектов должны быть разными, следовательно $s_i$ должно на объектах с большим штрафом хотеть больше уменьшить ошибку, так как это внесет большой вклад в ошибку фукнции потреь. Следовательно, мы хотим получить более крупные корректировки для объектво, которые в правой ветке функции потерь, но если использовать текущий вариант, тогда корректировка будет одинаковой для двух объектов, что плохо.

Более разумно потребовать, чтобы сдвиг $s_i$ был противоположен производной функции потерь в точке: $z = a_{N - 1}(x_i)$:

$\large s_i
=
-
\left.
\frac{\partial L}{\partial z}
\right|_{z = a_{N - 1}(x_i)}$

Мы придумали чему должны равняться корректировки, но это на данный момент просто числа, а нам необходимо обучать модели. Так как мы знаем, чему дожна быть равна корректировка в $L$ точках, тогда, обучим базовую модель так, чтобы она была как можно ближе к $s_i$. Если базовая модель будет равняться примено сдвигу, тогда мы сдвиним ошибку в меньшую сторону, так как $b_N$ будет двигать в ту сторону, в которую ошибка будешь уменьшаться. Следовательно, будем обучать очередную базовую модель на следующую задачу:

$\large \frac{1}{\ell}\sum\limits_{i = 1}^{\ell} (b_N(x_i) - s_i)^2 \to \underset{b_N(x)}{\text{min}}$

### Причина обучения на MSE

Почему не обучаем дерево на исходную фукнцию потерь?

$\large \frac{1}{\ell}\sum\limits_{i = 1}^{\ell} L(s_i, b_N(x_i)) \to \underset{b_N(x)}{\text{min}}$

Обучение на MSE может бы . Расскроем квадрат функционала ошибки:

$\large \frac{1}{\ell}\sum\limits_{i = 1}^{\ell} (b_N^2(x_i) - 2 b_N^2(x_i) s_i + s_i^2) \to \underset{b_N(x)}{\text{min}}$

Тогда $s_i^2$ - это конастанта, поэтому можно его выкинуть. $b_N^2(x_i)$ в некоторой степени регуляризатор. И остается $-2 b_N^2(x_i) s_i$, которое мы минимизируем, если убрать минус тогда это необходимо максимизировать. Это выражение является скалярным произведение выходов модели с остатками, то есть, это угол между направлением в котором необходимо сдвигать прогнозы композиции и направлением, которое дерево смогло выучить, и мы хотим чтобы эти направления были как можно сильнее соноправленными.

Получаем, что в бустинге две фукнции потерь, исходная, которая используется для рассчета градинета и ещё одна для обучения базовой модели на градиент.

### Почему это называется градиентным бустингом?

Предположим, что в выборке всего два объекта 

* $\ell = 2$

* $X = \{(x_1, y_1), (x_2, y_2)\}$

Будем на этой выборке обучать бустинг, тогда по оси $x$ будет прогноз на первом объекте $a_N(x_1)$, на оси $y$ будет нарисован прогноз текущей композиции на втором объекте $a_N(x_2)$. И нарисуем линии уровня ошибки этой модели, например, $L(y_1, a_N(x_1)) + L(y_2, a_N(x_2))$.

* Выбираем первую константную модель и ставим точку на графике. 

* Считаем сдвии, то есть, считаем градиент нашего фукнционала $L(y_1, a_N(x_1)) + L(y_2, a_N(x_2))$, по  параметрам $a_N(x_1), a_N(x_2)$. Следовательно, мы считаем куда необходимо менять прогнозы на объектах, чтобы ошибка уменьшилась как можно сильнее. После рассчетов, перемещаемся на графике в сторону сдвига.

* Подбираем модель $b_2$ такой, чтобы она аппроксимировала полученное направление. Дальше мы строим модель и прибавляем к композиции, так как она довльно простая, то сдвинимся немного в другую сторону, переместившись в следующую точку.

* После в этой точке считаем градиент, аппроксимиров его следующей моделью

* ...

Получается, что на каждом шаге мы считаем куда необходимо сдвинуть прогноз композии, аппроксимируем это направление и двигаемся в его направлении. И итоговая композиция моделей $a_N$ приходит в точку оптимума.

Следовательно, градиентный бустинг - это градиентый спуск в пространстве прогнозов композиции на обучающей выборке. 

<h1 style="color:#008B8B">3. Регуляризация</h1>

### 3.1 Сокращение шага

На практике оказывается, что градиентный бустинг очень быстро строит композицию, ошибка которой на обучении выходит на асимптоту, после чего начинает настраиваться на шум и переобучаться. Это явление можно объяснить одной из двух причин:

*  Если базовые алгоритмы очень простые (например, решающие деревья небольшой глубины), то они плохо приближают вектор антиградиента. По сути, добавление такого базового алгоритма будет соответствовать шагу вдоль направления, сильно отличающегося от направления наискорейшего убывания. Соответственно, градиентный бустинг может свестись к случайному блужданию в пространстве.
* Если базовые алгоритмы сложные (например, глубокие решающие деревья), то они способны за несколько шагов бустинга идеально подогнаться под обучающую выборку — что, очевидно, будет являться переобучением, связанным с излишней сложностью семейства алгоритмов.


Хорошо зарекомендовавшим себя способом решения данной проблемы является _сокращение шага_: вместо перехода в оптимальную точку в направлении антиградиента делается укороченный шаг:


$\large a_N(x)
=
a_{N - 1}(x)
+
\eta
b_N(x),$

где $\eta \in (0, 1]$ - темп обучения. Как правило, чем меньше темп обучения, тем лучше качество итоговой композиции. Идея заключатеся в том, что при небольшом размере шага направление движения болоее стабильное, что более вероятно приводит к минимуму фукнции. Сокращение шага, по сути, позволяет понизить доверие к направлению, восстановленному базовым алгоритмом.

Также следует обратить внимание на число итераций градиентного бустинга. В отличии от бэггинга, градиентный бустинг переобучается с ростом $N$. Поэтому необходимо подбирать $N$ по отложенной выборке. Хотя ошибка на обучении монотонно стремится к нулю, ошибка на контроле, как правило, начинает увеличиваться после определенной итерации. Оптимальное число итераций можно выбирать, например, по отложенной выборке или с помощью кросс-валидации.

### 3.2 Стохастический градиентный бустинг

Еще одним способом улучшения качества градиентного бустинга является внесение рандомизации в процесс обучения базовых алгоритмов. А именно, алгоритм $b_N$ обучается не по всей выборке $X$, а лишь по ее случайному подмножеству $X^k \subset X$. В этом случае понижается уровень шума в обучении, а также повышается эффективность вычислений. Уровень шума понижается из-за того, что слабой модели тяжело уловить зависимости сразу на огромном количестве объектов. Существует рекомендация брать подвыборки, размер которых вдвое меньше исходной выборки.

<h1 style="color:#008B8B">4. Функции потерь</h1>

### 4.1 Регрессия

При вещественном целевом векторе, как правило, используют квадратичную функцию потерь, формулы для которой уже были приведены в разделе 1. Другой вариант - модуль отклонения $L(y, z) = |y - z|$, для которого антиградиент вычисляется по формуле:

$\large s_i^{(N)}
=
-
\text{sign}
(a_{N - 1}(x_i) - y_i).$

### 4.2 Классификация

Базовые модели обучаются на задачу регрессии MSE, но в MSE сдвиги $s_i$ зависят от исходной фукнции потерь, следовательно, в задаче обучения базовой модели на MSE входит информация о исходной фукнции потерь. И исходную фукнцию потерь для композиции мы задаем сами, указав тем самым, что требуется от композиции. 

Что модели должны выдавать? Пусть модель $b_n(x)$ выдает число, которое мы интерпретируем как уверенность в том, что $x$ относится к положительному классу. Где $b_n(x) \in \mathbb{R}$.

Если базовые модели выдают уверенность, тогда композиция - это сумма базовых моделей, следовательно, композицию тоже можно интерпретировать как уверенность:

$\large a_N(x)
=
\sum\limits_{n = 1}^{N}
    b_n(x)$
    
Какие фукнции потерь штрафуют за плохую уверенность в классификации? В задаче классификации с двумя классами разумным выбором является логистическая функция потерь, с которой уже сталкивались при изучении линейных методов:

$\large L(y, z)
=
\log(
    1 + \exp(-yz)
).$

Эта фукнция потерь требует, чтобы $z$ было как можно больше, так как минимум логистической фукнции потерь достигается при выходе модели $z$ равном бесконечности. Следовательно, эта фукнция потерь хорошо подходит, такак как работает с вещественными выходами модели, только в классификации.

Запишем сдвиги для логистической функции потерь:

$\large s_i = \frac{
        y_i
    }{
        1 + \exp(
            y_i a_{N - 1}(x_i)
        )
    }$

* Допустим, на некотором объекте отступ равен примерно плюс бесконечности $y_i a_{N-1}(x_i) \approx +\infty$, тогда знаменатель будет примерно плюс бесконечность и сдвиг будет равен нулю $s_i \approx 0$. Следовательно, если модель на данном объекте имеет высокую уверенность в правильном ответе, тогда следующая модель не должна использовать данный объект.

* Допустим, на некотором объекте отступ примерно равен нулю $y_i a_{N-1}(x_i) \approx 0$, тогда в знаменатель подставим $y_i a_{N-1}(x_i)$ почти равное нулю и сдвиги по модулю будут примерно равны $|s_i| \approx \frac{1}{2}$.
 
* Допустим, на некотором объекте отступ равен примерно минус бесконечности $y_i a_{N-1}(x_i) \approx -\infty$, тогда сдвиг будет равен примерно еденице $|s_i| \approx 1$.

Если отступ большой и отрицательный, но модель уверена в ответе, а овтет неправлиьный, тогда, скорее всего это выброс. Тогда, сдвиг будет не больше, чем на нормальном объекте, на котором модель ошибается. Делаем вывод, что логистическая фукнция потерь устойчива к выборосам.

Задача поиска базового алгоритма с ней принимает вид:

$\large b_N
=
\underset{b \in \mathbb{A}}{\text{argmin}}
\sum\limits_{i = 1}^{\ell} \left(
    b(x_i)
    -
    \frac{
        y_i
    }{
        1 + \exp(
            y_i a_{N - 1}(x_i)
        )
    }
\right)^2.$

Так же можно использовать другие фукцнии потерь, например hinge-loss - в этом случае мы не получим уверенность модели в некотором классе, может оно нам и не надо.

<h1 style="color:#008B8B">5. Градиентный бустинг над деревьями (GBDT)</h1>

Считается, что градиентный бустинг над решающими деревьями — один
из самых универсальных и сильных методов машинного обучения, известных на сегодняшний день. В частности, на градиентном бустинге над деревьями основан MatrixNet — алгоритм ранжирования компании Яндекс.

Мы строим бустинг, где базовые моделеи это деревья, если базовые модели это деревья, тогда возникает один очень важный момент, который позволяет строить композиции намного лучше, чем в подходе выше. Вспомним, что решающее дерево разбивает все пространство на непересекающиеся области, в каждой из которых его ответ равен константе:

$\large b_n(x)
=
\sum\limits_{j = 1}^{J_n}
    b_{nj}
    [x \in R_{nj}],$

где $j = 1, \dots, J_n$ - индексы листьев, $R_{nj}$ - соответствующие области разбиения, $b_{nj}$ - значения в листьях. Посмторим, как дерево добавляется в композицию, на $N$-й итерации бустинга композиция обновляется как

$\large a_N(x)
=
a_{N - 1}(x)
+
\eta
\sum\limits_{j = 1}^{J_N}
    b_{Nj}
    [x \in R_{Nj}]
=$ 

### НАПИСАТЬ ИДЕЮ ФОРМУЛЫ И ПРОЦЕСС ДООБУЧЕНИЯ

В этой формуле после дилны шага должна стоять $\gamma$, но мы уже говорили почему ее не используем. $\eta$ тоже можно выбросить, так как мы продолжаем обучать деревья, а идея использования шага заключается в том, что готовую модель добавляем в композицию с некоторым весом. Посскольку мы продолжаем дообучение модели, поэтму $\eta$ убираем, добавим ее когда модель дообучим и решим добавить ее в композицию.
    
**Смещение и разброс.** В случайных лесах используются глубокие деревья, поскольку от базовых алгоритмов требуется низкое смещение; разброс же устраняется за счёт усреднения ответов различных деревьев.

Почему смещение уменьшается? Имеется некоторая базовая модель (например, решающий пень), втору модель добавляем так, чтобы она корректировала ошибки первой модели, следовательно, вторая модель знает о превой и подстраивается так, чтобы делать лучше. И эти две модели работают совместно, их параметры подбираются так, чтобы улучшать общую ошибку. Поэтому смещение уменьшается, за счет того, что композиция работает как одна модель, мы параметры подбираем совместно. В бэгинге смещение не менялось, так как каждая модель была независимой, добавля очередную модель, она ничего не знает о предыдыщей, следовательно переобучить уже построенную композицию она не может, а в бустинге новая модель может переобучить построенную композицию за счет корректировки.

Разброс же возрастате из-за того, что параметров становится много и чем больше параметров, тем модель больше способна к переобучению, следоватлеьно, становится более чувствительная к изменению в данных.

Из-за этого, как правило, в бустинге используются неглубокие решающие деревья (3-6 уровней), которые обладают большим смещением, но не склонны к переобучению.